In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from sklearn.preprocessing import LabelBinarizer
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD
from keras.models import load_model
from sklearn.metrics import classification_report
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet50
from imutils import paths
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pickle

In [4]:
#dataset='/content/gdrive/MyDrive/image'
dd='/content/gdrive/MyDrive/data_p'
LABELS = set(["fabric", "foliage", "glass","leather","metal"," paper","plastic","stone","water"," wood"])
imagePaths = list(paths.list_images(dd))

data1 = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
 # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
# if the label of the current image is not part of the labels
 # are interested in, then ignore the image
    if label not in LABELS:
         continue
# load the image, convert it to RGB channel ordering, and resize
 # it to be a fixed 224x224 pixels, ignoring aspect ratio
    image = cv2.imread(imagePath)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#RGB images
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)#gray images

    image = cv2.resize(image, (16, 16))
# update the data and labels lists, respectively
    data1.append(image)
    labels.append(label)

In [13]:
print(data1.shape)
print(labels.shape)

(800, 16, 16)
(800, 8)


In [13]:
data=data1.reshape((800, 16*16))

In [11]:
# convert the data and labels to NumPy arrays
data1 = np.array(data1)
labels = np.array(labels)  
#Encode labels from text (folder names) to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#labels1=le.fit(labels)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [14]:
# #GLCM feature extraction

import numpy as np
from skimage import feature, io
from sklearn import preprocessing

#img = io.imread(data1, as_grey=True)

S = preprocessing.MinMaxScaler((0,11)).fit_transform(data).astype(int)
Grauwertmatrix = feature.greycomatrix(S, [1,2,3], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=12, symmetric=False, normed=True)

ContrastStats = feature.greycoprops(Grauwertmatrix, 'contrast')
CorrelationtStats = feature.greycoprops(Grauwertmatrix, 'correlation')
HomogeneityStats = feature.greycoprops(Grauwertmatrix, 'homogeneity')
ASMStats = feature.greycoprops(Grauwertmatrix, 'ASM')

print([np.mean(ContrastStats), np.mean(CorrelationtStats),\
np.mean(ASMStats), np.mean(HomogeneityStats)])

[8.87501967691181, 0.23878017534929488, 0.019935118125526525, 0.40372057567235453]


In [16]:
n_img = len(data1)
print(n_img)

800


In [15]:
import numpy as np
from skimage.feature import greycomatrix, greycoprops

ANGLES = [0., np.pi/4., np.pi/2., 3.*np.pi/4.]
DISTANCES = [1,2]
properties = ["correlation", "contrast", "homogeneity", "energy"]
n_img = len(data1)
#data1 = np.random.randint(0,255,size=(n_img, 100, 100)),,,  data1.shape


stats = []
for k in range(n_img):
    #glcm = greycomatrix(I[k], distances=DISTANCES, angles=ANGLES, levels=256, symmetric=True, normed=True)
    glcm = greycomatrix(data1[k], distances=DISTANCES, angles=ANGLES, levels=256, symmetric=True, normed=True)

    prop = [np.mean(greycoprops(glcm, properties[i])) for i in range(len(properties))]
    stats.append(prop)
stats = np.array(stats)

In [17]:
print(stats.shape)
print(labels.shape)


(800, 4)
(800, 8)


In [21]:
(x_train, y_train, x_test, y_test) = train_test_split(S, labels,
 test_size=0.3, stratify=labels, random_state=42)

In [25]:
(x_train, y_train, x_test, y_test) = train_test_split(stats, labels,
 test_size=0.3, stratify=labels, random_state=0)

In [26]:
print(x_train.shape)
print(y_train.shape)

(560, 4)
(240, 4)


In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
  
knn = KNeighborsClassifier(n_neighbors = 1)
#y_train=y_train.reshape(-1,1)  
clf=knn.fit(x_train, y_train)
pred = knn.predict(x_test)
print(accuracy_score(pred,y_test))

ValueError: ignored